## Deep Learning¶
Assignment 4
Previously in 2_fullyconnected.ipynb and 3_regularization.ipynb, we trained fully connected networks to classify notMNIST characters.
The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from time import time
import math
%matplotlib inline

# First reload the data we generated in 1_notmnist.ipynb.
pickle_file = 'proj1/notMNIST_original.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions,1) == np.argmax(labels,1))
            / predictions.shape[0])

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [19]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
LR = 0.05

graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
     tf.float32,shape=(batch_size,image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(
        batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev = 0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev = 0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size//4 * image_size //4 * depth, num_hidden],stddev =0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding = 'SAME')
        hidden1 = tf.nn.relu(conv + layer1_biases)
        print('a batch of images input:', data.get_shape().as_list())
        print('weights1 shape:', layer1_weights.get_shape().as_list())
        print('conv with weights1 output:', conv.get_shape().as_list())
        print('biases1:', layer1_biases.get_shape().as_list())
        print('hidden1:', hidden1.get_shape().as_list())
        conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding = 'SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        print('conv2 input:', hidden1.get_shape().as_list())
        print('weights2 shape:', layer2_weights.get_shape().as_list())
        print('conv2 with weights2 output:', conv2.get_shape().as_list())
        print('biases2:', layer2_biases.get_shape().as_list())
        print('hidden2:', hidden2.get_shape().as_list())
        
        shape = hidden2.get_shape().as_list()
        reshape = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
        print('hidden2 is reshaped into:', reshape.get_shape().as_list())
        
        hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        print('hidden3 is genereated by a full connected NN:', hidden3.get_shape().as_list())
         
        hidden4 = tf.matmul(hidden3, layer4_weights) + layer4_biases
        print('hidden4 is generated by a full connected layer:', hidden4.get_shape().as_list())
        return(hidden4)
        
        
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
       tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(LR).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
        
    

a batch of images input: [16, 28, 28, 1]
weights1 shape: [5, 5, 1, 16]
conv with weights1 output: [16, 14, 14, 16]
biases1: [16]
hidden1: [16, 14, 14, 16]
conv2 input: [16, 14, 14, 16]
weights2 shape: [5, 5, 16, 16]
conv2 with weights2 output: [16, 7, 7, 16]
biases2: [16]
hidden2: [16, 7, 7, 16]
hidden2 is reshaped into: [16, 784]
hidden3 is genereated by a full connected NN: [16, 64]
hidden4 is generated by a full connected layer: [16, 10]
a batch of images input: [10000, 28, 28, 1]
weights1 shape: [5, 5, 1, 16]
conv with weights1 output: [10000, 14, 14, 16]
biases1: [16]
hidden1: [10000, 14, 14, 16]
conv2 input: [10000, 14, 14, 16]
weights2 shape: [5, 5, 16, 16]
conv2 with weights2 output: [10000, 7, 7, 16]
biases2: [16]
hidden2: [10000, 7, 7, 16]
hidden2 is reshaped into: [10000, 784]
hidden3 is genereated by a full connected NN: [10000, 64]
hidden4 is generated by a full connected layer: [10000, 10]
a batch of images input: [10000, 28, 28, 1]
weights1 shape: [5, 5, 1, 16]
conv with

In [20]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    # Use tf.initialize_all_variables() to add an op to run 
    # variable initializers.
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
               valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.056650
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.722548
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 0.506272
Minibatch accuracy: 87.5%
Validation accuracy: 82.6%
Test accuracy: 89.3%


## Problem 1
The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.


In [29]:
batch_size = 16
patch_size = 2
num_hidden = 64
depth =1
LR = 0.05

graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
     tf.float32,shape=(batch_size,image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(
        batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size//4 * image_size //4 * depth, num_hidden],stddev =0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    -
        
        
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
       tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(LR).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
        
    

a batch of images input: [16, 28, 28, 1]
max_pool1 output: [16, 14, 14, 1]
hidden1: [16, 14, 14, 1]
max_pool2 input: [16, 14, 14, 1]
max_pool2 output: [16, 7, 7, 1]
hidden2: [16, 7, 7, 1]
hidden2 is reshaped into: [16, 49]
hidden3 is genereated by a full connected NN: [16, 64]
hidden4 is generated by a full connected layer: [16, 10]
a batch of images input: [10000, 28, 28, 1]
max_pool1 output: [10000, 14, 14, 1]
hidden1: [10000, 14, 14, 1]
max_pool2 input: [10000, 14, 14, 1]
max_pool2 output: [10000, 7, 7, 1]
hidden2: [10000, 7, 7, 1]
hidden2 is reshaped into: [10000, 49]
hidden3 is genereated by a full connected NN: [10000, 64]
hidden4 is generated by a full connected layer: [10000, 10]
a batch of images input: [10000, 28, 28, 1]
max_pool1 output: [10000, 14, 14, 1]
hidden1: [10000, 14, 14, 1]
max_pool2 input: [10000, 14, 14, 1]
max_pool2 output: [10000, 7, 7, 1]
hidden2: [10000, 7, 7, 1]
hidden2 is reshaped into: [10000, 49]
hidden3 is genereated by a full connected NN: [10000, 64]
h

In [32]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    # Use tf.initialize_all_variables() to add an op to run 
    # variable initializers.
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
               valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.544172
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.686011
Minibatch accuracy: 87.5%
Validation accuracy: 72.8%
Minibatch loss at step 2000: 0.342465
Minibatch accuracy: 93.8%
Validation accuracy: 76.9%
Minibatch loss at step 3000: 0.604201
Minibatch accuracy: 81.2%
Validation accuracy: 75.7%
Minibatch loss at step 4000: 0.773452
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Minibatch loss at step 5000: 0.937630
Minibatch accuracy: 62.5%
Validation accuracy: 77.2%
Test accuracy: 85.0%


##  Problem 2
Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.